# AI201 - Programming Assignment 1
### Implementing the A* Algorithm
*Instructor:	Pros Naval*<br>
*Submitted by:  Mike Allan Nillo*

#### A* Pseduo-code
1. Put the start node s on a list called OPEN and compute f(s).
2. If OPEN is empty, exit with failure; otherwise continue.
3. Remove from OPEN that node whose f value is smallest and put it on a list called CLOSED. Call this node n. (Resolve ties for minimal f values arbitrarily, but always in favor of any goal node.)
4. If n is a goal node, exit with the solution path obtained by tracing back the pointers; otherwise continue.
5. Expand node n, generating all of its successors. If there are no successors, go immediately to 2. For each successor ni, compute f(n).
6. Associate with the successors not already on either OPEN or CLOSED the ƒ values just computed. Put these nodes on OPEN and direct pointers from them back to n.
7. Associate with those successors that were already on OPEN or CLOSED the smaller of the ƒ values just computed and their previous f values. Put on OPEN those successors on CLOSED whose ƒ values were thus lowered, and redirect to n the pointers from all nodes whose ƒ values were lowered.
8. Go to 2.

*Comments:*
- duplicates are not retained; when nodes are rediscovered, the ancestor history is updated
- when a successor is already on OPEN or CLOSED, the algorithm modifies the pointers so that the nodes record the shorter of the two partial paths

### Defining Utility Functions

In [1]:
class Node:
    def __init__(self, state, parent=None, g=0, h=0, p_n=0, s_n=0):
        self.state = state
        self.parent = parent
        self.g = g
        self.h = h
        self.p_n = p_n
        self.s_n = s_n
        self.f = self.g + self.h

    def __lt__(self, other):
        return self.f < other.f

def read_astar_input(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        start_index = lines.index('start\n') + 1
        goal_index = lines.index('goal\n') + 1
        start_state = [list(map(int, line.replace('*', '0').split())) for line in lines[start_index:start_index+3]]
        goal_state = [list(map(int, line.replace('*', '0').split())) for line in lines[goal_index:goal_index+3]]
    return start_state, goal_state

def get_neighbors(state):
    # Find the 0 tile
    i, j = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]

    # List of possible moves
    moves = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

    # Neighbors list
    neighbors = []

    # For each possible move
    for move in moves:
        # If the move is within the bounds of the puzzle
        if 0 <= move[0] < 3 and 0 <= move[1] < 3:
            # Copy the current state to create a new state
            new_state = [row.copy() for row in state]
            # Swap the 0 tile with the neighboring tile
            new_state[i][j], new_state[move[0]][move[1]] = new_state[move[0]][move[1]], new_state[i][j]
            # Add the new state to the list of neighbors
            neighbors.append(new_state)

    return neighbors

def misplaced_tiles(start, goal):
    h = sum(start[i][j] != goal[i][j] for i in range(3) for j in range(3))
    return h, 0, 0

def manhattan_distance(start, goal):
    h = sum(abs(b % 3 - g % 3) + abs(b//3 - g//3)
             for b, g in ((start[i][j], goal[i][j])
                          for i in range(3) for j in range(3))
             if g != 0)
    return h, 0, 0

def nilssons_sequence(state, goal_state):
    p_n = 0
    s_n = 0
    h = 0

    # Define the correct sequence of tiles around the edges
    correct_sequence = [1, 2, 3, 8, 0, 4, 7, 6, 5]

    # Flatten the state and goal_state arrays
    state = [item for sublist in state for item in sublist]
    goal_state = [item for sublist in goal_state for item in sublist]

    # Create a list of the current sequence of tiles around the edges
    current_sequence = [state[correct_sequence.index(i)] for i in range(1, 9)]

    # Calculate S(n)
    for i in range(8):  # We only go up to 8 because the last tile wraps around to the first
        if current_sequence[(i+1)%8] != correct_sequence[(correct_sequence.index(current_sequence[i])+1)%8]:
            s_n += 2

    # Calculate P(n)
    for i in range(9):
        if state[i] != goal_state[i]:
            p_n += abs(i // 3 - goal_state.index(state[i]) // 3) + abs(i % 3 - goal_state.index(state[i]) % 3)

    # Calculate h(n)
    h = p_n + 3 * s_n

    return h, p_n, s_n

def print_path(node):
    path = []
    while node is not None:
        path.append(node)
        node = node.parent
    path.reverse()
    for node in path:
        print(node.state)
        print(f"f(n): {node.f}, g(n): {node.g}, h(n): {node.h}")
        print(f"P(n): {node.p_n}")
        print(f"S(n): {node.s_n}")
        print()

### A* Algorithm Implementation

In [2]:
def a_star(start, goal, heuristic):
    open_list = []
    closed_list = set()

    # Put the start node s on a list called OPEN and compute f(s)
    h, p_n, s_n = heuristic(start, goal)
    start_node = Node(start, h=h, p_n=p_n, s_n=s_n)
    open_list.append(start_node)

    # If OPEN is empty, exit with failure; otherwise continue.
    while open_list:
        # Sort the open list and pop the node with the lowest f score
        open_list.sort(key=lambda node: node.f)
        current = open_list.pop(0)

        # If the current node is the goal, return the path
        if current.state == goal:
            return current, len(closed_list)

        closed_list.add(tuple([item for sublist in current.state for item in sublist]))

        for neighbor_state in get_neighbors(current.state):
            neighbor_tuple = tuple([item for sublist in neighbor_state for item in sublist])

            if neighbor_tuple in closed_list:
                continue

            h, p_n, s_n = heuristic(neighbor_state, goal)
            neighbor = Node(neighbor_state, current, current.g + 1, h, p_n, s_n)

            # If the neighbor is already in the open list, update its f value
            for node in open_list:
                if node.state == neighbor_state and node.f > neighbor.f:
                    node.f = neighbor.f
                    node.parent = current
                    break
            else:
                open_list.append(neighbor)

    return None, len(closed_list)

### Testing A* Implementation to `astar_in.txt`

In [3]:
# read input file
start_state, goal_state = read_astar_input('astar_in.txt')

In [4]:
# Test using: Number of Tiles in the Wrong Position
goal_node, search_cost = a_star(start_state, goal_state, misplaced_tiles)
print('Misplaced Tiles:')
print_path(goal_node)
print('Search cost:', search_cost)
print('===========')

# Test using: Manhattan Distance
goal_node, search_cost = a_star(start_state, goal_state, manhattan_distance)
print('Manhattan Distance:')
print_path(goal_node)
print('Search cost:', search_cost)
print('===========')

#Test using: Nilssons Sequence Score
goal_node, search_cost = a_star(start_state, goal_state, nilssons_sequence)
print('Nilsson\'s Sequence Score:')
print_path(goal_node)
print('Search cost:', search_cost)
print('===========')

Misplaced Tiles:
[[2, 1, 6], [4, 0, 8], [7, 5, 3]]
f(n): 7, g(n): 0, h(n): 7
P(n): 0
S(n): 0

[[2, 1, 6], [4, 8, 0], [7, 5, 3]]
f(n): 9, g(n): 1, h(n): 8
P(n): 0
S(n): 0

[[2, 1, 0], [4, 8, 6], [7, 5, 3]]
f(n): 10, g(n): 2, h(n): 8
P(n): 0
S(n): 0

[[2, 0, 1], [4, 8, 6], [7, 5, 3]]
f(n): 11, g(n): 3, h(n): 8
P(n): 0
S(n): 0

[[2, 8, 1], [4, 0, 6], [7, 5, 3]]
f(n): 11, g(n): 4, h(n): 7
P(n): 0
S(n): 0

[[2, 8, 1], [4, 6, 0], [7, 5, 3]]
f(n): 13, g(n): 5, h(n): 8
P(n): 0
S(n): 0

[[2, 8, 1], [4, 6, 3], [7, 5, 0]]
f(n): 14, g(n): 6, h(n): 8
P(n): 0
S(n): 0

[[2, 8, 1], [4, 6, 3], [7, 0, 5]]
f(n): 14, g(n): 7, h(n): 7
P(n): 0
S(n): 0

[[2, 8, 1], [4, 0, 3], [7, 6, 5]]
f(n): 13, g(n): 8, h(n): 5
P(n): 0
S(n): 0

[[2, 8, 1], [0, 4, 3], [7, 6, 5]]
f(n): 15, g(n): 9, h(n): 6
P(n): 0
S(n): 0

[[0, 8, 1], [2, 4, 3], [7, 6, 5]]
f(n): 16, g(n): 10, h(n): 6
P(n): 0
S(n): 0

[[8, 0, 1], [2, 4, 3], [7, 6, 5]]
f(n): 17, g(n): 11, h(n): 6
P(n): 0
S(n): 0

[[8, 1, 0], [2, 4, 3], [7, 6, 5]]
f(n): 18, g(n